In [49]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.common.exceptions import NoSuchWindowException   
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os
import sys

In [50]:
download_dir = "C:\\Users\\TFD\\PDF" # for linux/*nix, download_dir="/usr/Public"
options = webdriver.ChromeOptions()
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)
driver = webdriver.Chrome("C:\\Users\\TFD\\Anaconda3\\chromedriver.exe", chrome_options=options)  # Optional argument, if not specified will search path.
ActionChainsDriver = ActionChains(driver)
wait = WebDriverWait(driver,10)

C:\Users\TFD\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [51]:
driver.maximize_window()
driver.get("https://ssopxy.gov.taipei/SSOPXY/Default.aspx")
id =('j0470@tfd.gov.tw')
password = ('j180065@')

In [52]:
driver.find_element_by_name('TextBox_userID').send_keys(id)

In [53]:
driver.find_element_by_name('TextBox_userPwd').send_keys(password)

In [54]:
driver.find_element_by_name('Button_Login').click()

In [55]:
driver.find_element_by_class_name('category-button1').click()

In [56]:
driver.find_element_by_css_selector('input[value=新公文系統]').click()

In [57]:
child = driver.window_handles[1]      
driver.switch_to.window(child)
print(child)

CDwindow-554C26CB90C54840DAC6197460AEA9CA


In [58]:
driver.implicitly_wait(30)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "toSaveCheck"))   
        
    )
finally:
    driver.find_element_by_id('toSaveCheck').click()

In [59]:
sleep(1)
driver.find_element_by_id('menu1487').click()

In [60]:
print (driver.current_url)

https://doc.gov.taipei/tcqb/home/default.jsp#


In [61]:
driver.switch_to.frame('dTreeContent')
driver.find_element_by_name('conditionQueryAll').click()

In [62]:
sleep(2)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q_IO_DEP1_NO"))   
    )
finally:
    driver.find_element_by_xpath("//select[@name='q_epaper2']/option[text()='線上']").click()

In [63]:
#"BDAW-第一救災救護大隊"
#"BDAX-第二救災救護大隊"
#"BDAY-第三救災救護大隊"
#"BDAZ-第四救災救護大隊"
driver.find_element_by_xpath("//select[@name='q_IO_DEP1_NO']/option[@value='BDAX']").click()
driver.find_element_by_xpath("//*[@id='qryModal']/div[2]/div/div[2]/div/div/div/table/tbody/tr[3]/td[4]/select/option[text()='電子收文']").click()

In [64]:
x = "貴公司"
driver.find_element_by_name('q_ABSTRACT').clear()
driver.find_element_by_name('q_ABSTRACT').send_keys(x)

In [65]:
driver.find_element_by_name('querySubmit').click()

In [66]:
def select_document():
    sleep(1)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[1]/td[1]').click()
    driver.find_element_by_name('000091902').click()

In [67]:
def edit_case_number():
    driver.find_element_by_css_selector('#u_caseno').clear()
    driver.find_element_by_css_selector('#u_portfoliono').clear()
    driver.find_element_by_xpath('//*[@id="query"]').click()
    sleep(2)
    child2 = driver.window_handles[2]
    driver.switch_to.window(child2)
    driver.find_element_by_css_selector('#listTBODY > tr:nth-child(4)').click()

In [68]:
def get_PDF_string():
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[5]/td[2]/input[3]').click()
    child3 = driver.window_handles[2]
    driver.switch_to.window(child3)
    sleep(3)
    def get_PDF_url():
        source = driver.page_source
        startPDF_url=source.find('blob:https://doc.')
        namePDF_url =source.find('blob:https://doc.gov.taipei/')
        EndPDF_url = source.find('">',startPDF_url)
        PDF_url =source[startPDF_url:EndPDF_url]
        PDF_name = source[startPDF_url+28:EndPDF_url]
        print(PDF_url)
        print(PDF_name)
        driver.get(PDF_url)
        sleep(3)
        return PDF_name
        
    def convert_pdf_to_txt(path):
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()
        return text
    try:
        sleep(2)
        p = get_PDF_url()
    except:
        sleep(2)
        p = get_PDF_url()
    PDF_path = "PDF\\"+ p + ".pdf"
    return convert_pdf_to_txt(PDF_path)

In [69]:
def edit_case_name_by_PDF():
    companyName = pdfText[pdfText.find('正本：')+3:pdfText.find('副本：')]
    print(companyName)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    # 獲取原始案由
    text = driver.find_element_by_id('u_abstractMain').get_attribute('value')
    print(text)
    #處理貴公司字串
    text = text.replace('貴公司',companyName)
    #處理查照字串
    x= text.find("，復請查照")
    if x == - 1:
        x=text.find("，請查照")
        if x == -1:
            print(text)
        else :
            text = text.replace(text[x:x+4],"")
            print(text)
    else:
        text = text.replace(text[x:x+5],"")
        print(text)
    print(text)
    #處理統一編號
    x=text.find("【分公司")
    if x == - 1:
        x=text.find("（統一編號")
        if x == -1:
            print(text)
        else :
            text = text.replace(text[x:x+15],"")
            print(text)
    else:
        text = text.replace(text[x:x+18],"")
        print(text)
    print(text)
    #處理隨案附告
    x=text.find("另隨案附告")
    if x == -1:
        x = text.find('隨案附告')
        if x == - 1:
            print(text)
        else:
            text = text.replace(text[x:len(text)],"")
            print(text)
    else:
        text = text.replace(text[x:len(text)],"")
    #處理句號後冗長語句
    x= text.find('。')
    if x +1 != len(text):
        text = text.replace(text[x:len(text)-1],"")
        print(text)
    else:
        print(text)

    driver.find_element_by_id('u_abstractMain').clear()
    driver.find_element_by_id('u_abstractMain').send_keys(text)

In [70]:
def check_number_of_attachment():
    driver.switch_to.window(driver.window_handles[2])
    driver.switch_to.default_content()
    source = driver.page_source
    s = source.find('共')
    y = source.find('件',s)
    if s != -1 and y != -1 :
        driver.switch_to.window(driver.window_handles[1])
        driver.switch_to.frame('dTreeContent')
        driver.find_element_by_name('u_mediunQty').clear()
        driver.find_element_by_name('u_mediunQty').send_keys(source[s+1:y])
        driver.switch_to.window(driver.window_handles[2])
        print("已修改為"+source[s+1:y]+"份")
    else:
        print('沒有附件')
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')


In [71]:
def check_insert_attach():
    driver.find_element_by_id('insertAttach').click()
    driver.switch_to.window(driver.window_handles[2])
    s = driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td').get_attribute('innerHTML')
    if s != "查無資料，請您重新輸入查詢條件！":
        driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[2]').click()
        driver.find_element_by_xpath('//*[@id="spanDelete"]/input').click()
        driver.switch_to.alert.accept()
        driver.switch_to.window(driver.window_handles[2])
        print("已將登陸附件刪除")
    else: 
        print("確認無登錄附件")
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')

In [ ]:
t = 1
for x in range (40):
    select_document()
    edit_case_number()
    pdfText = get_PDF_string()
    edit_case_name_by_PDF()
    check_number_of_attachment()
    check_insert_attach()
    driver.find_element_by_id('doConfirm').click()
    driver.find_element_by_id('close').click()
    print('已編目'+str(t)+'筆公文')
    print("--------------------------------------------------------分隔線--------------------------------------------------------")
    t += 1

    

blob:https://doc.gov.taipei/462b8f33-9712-45cd-baec-af719b9de09f
462b8f33-9712-45cd-baec-af719b9de09f
慶次國際開發有限公司
貴公司申請設立登記，准予登記（統一編號：50869997），隨案附告營業場所查詢結果，請務必詳閱說明三、四、五相關事項，以保權益。
慶次國際開發有限公司申請設立登記，准予登記（統一編號：50869997），隨案附告營業場所查詢結果，請務必詳閱說明三、四、五相關事項，以保權益。
慶次國際開發有限公司申請設立登記，准予登記（統一編號：50869997），隨案附告營業場所查詢結果，請務必詳閱說明三、四、五相關事項，以保權益。
慶次國際開發有限公司申請設立登記，准予登記，隨案附告營業場所查詢結果，請務必詳閱說明三、四、五相關事項，以保權益。
慶次國際開發有限公司申請設立登記，准予登記，隨案附告營業場所查詢結果，請務必詳閱說明三、四、五相關事項，以保權益。
慶次國際開發有限公司申請設立登記，准予登記，
慶次國際開發有限公司申請設立登記，准予登記，
沒有附件
確認無登錄附件
已編目1筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
blob:https://doc.gov.taipei/9a015661-8355-4825-8ca3-761f05906e26
9a015661-8355-4825-8ca3-761f05906e26
錦上添餐飲文化股份有限公司
更正本府107年8月9日府產業商字第10751956410號函核准貴公司（統一編號：42798979）改選董事、監察人、改選林佳穎為董事長、公司遷址變更登記處分函說明三，如本函說明二，請查照。
更正本府107年8月9日府產業商字第10751956410號函核准錦上添餐飲文化股份有限公司（統一編號：42798979）改選董事、監察人、改選林佳穎為董事長、公司遷址變更登記處分函說明三，如本函說明二。
更正本府107年8月9日府產業商字第10751956410號函核准錦上添餐飲文化股份有限公司（統一編號：42798979

In [ ]:
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()
os.execl(sys.executable, os.path.abspath('編目final.ipynb'), *sys.argv) 